# NEON Foliar, Root, Litter Stoichiometry analysis

In this notebook, I will analyze foliar CNP data previously collected by NEON, alongside previously collected litter and root CN data and newly collected litter and root P data to look for patterns and links in plant-level stoichiometry across species and sites. 

### Datasets needed: 
 - P data from lab analyses, spring 2022
 - NEON database foliar data
 - NEON database litter and root CN data
 - NEON database site variables: Soil data; climate vars like MAT, MAP; species data

In [11]:
## Install packages
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
base = importr('base')
utils = importr('utils')
stats = importr('stats')
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
import logging
rpy2_logger.setLevel(logging.ERROR)
utils.install_packages('neonUtilities', repos='https://cran.rstudio.com/');
neonUtilities = importr('neonUtilities')


The downloaded binary packages are in
	/var/folders/_g/j5dfnhqx19v1g0pvv_1nx_3c0000gn/T//RtmpDMWBg7/downloaded_packages


In [15]:
##Import and stack foliar data
neonUtilities.stackByTable(filepath='/Users/emmahauser/OneDrive - The University of Montana/NEON_LeafLitterAnalyses/NEON-Data/NEON_traits-foliar-2');

Stacking operation across a single core.
Stacking table cfc_carbonNitrogen
  |                                                  | 0 % ~calculating   |++++                                              | 6 % ~00s           |+++++++                                           | 12% ~00s           |++++++++++                                        | 19% ~00s           |+++++++++++++                                     | 25% ~00s           |++++++++++++++++                                  | 31% ~00s           |+++++++++++++++++++                               | 38% ~00s           |++++++++++++++++++++++                            | 44% ~00s           |+++++++++++++++++++++++++                         | 50% ~00s           |+++++++++++++++++++++++++++++                     | 56% ~00s           |++++++++++++++++++++++++++++++++                  | 62% ~00s           |+++++++++++++++++++++++++++++++++++               | 69% ~00s           |++++++++++++++++++++++++++++++++++++++            | 75% ~0

  |                                                  | 0 % ~calculating   |++++                                              | 6 % ~00s           |+++++++                                           | 12% ~00s           |++++++++++                                        | 19% ~00s           |+++++++++++++                                     | 25% ~00s           |++++++++++++++++                                  | 31% ~00s           |+++++++++++++++++++                               | 38% ~00s           |++++++++++++++++++++++                            | 44% ~00s           |+++++++++++++++++++++++++                         | 50% ~00s           |+++++++++++++++++++++++++++++                     | 56% ~00s           |++++++++++++++++++++++++++++++++                  | 62% ~00s           |+++++++++++++++++++++++++++++++++++               | 69% ~00s           |++++++++++++++++++++++++++++++++++++++            | 75% ~00s           |+++++++++++++++++++++++++++++++++++++++++         | 81% ~00s 

In [16]:
#Reading stacked files
import os
os.listdir('/Users/emmahauser/OneDrive - The University of Montana/NEON_LeafLitterAnalyses/NEON-Data/NEON_traits-foliar-2/stackedFiles/')

['cfc_LMA.csv',
 'validation_10026.csv',
 'variables_10026.csv',
 'cfc_elements.csv',
 'readme_10026.txt',
 'cfc_fieldData.csv',
 'cfc_carbonNitrogen.csv',
 'cfc_chlorophyll.csv',
 'issueLog_10026.csv',
 'cfc_lignin.csv',
 'categoricalCodes_10026.csv',
 'cfc_chemistrySubsampling.csv']

In [18]:
## Read in and look at C,N,P foliar data
import pandas
elements = pandas.read_csv('/Users/emmahauser/OneDrive - The University of Montana/NEON_LeafLitterAnalyses/NEON-Data/NEON_traits-foliar-2/stackedFiles/cfc_elements.csv')
CNisotopes = pandas.read_csv('/Users/emmahauser/OneDrive - The University of Montana/NEON_LeafLitterAnalyses/NEON-Data/NEON_traits-foliar-2/stackedFiles/cfc_carbonNitrogen.csv')

In [20]:
CNisotopes

,uid,namedLocation,domainID,siteID,plotID,plotType,sampleID,sampleCode,cnSampleID,cnSampleCode,...,analyticalRepNumber,remarks,laboratoryName,testMethod,instrument,analyzedBy,reviewedBy,dataQF,publicationDate,release
0,c6ec17bd-abb6-485f-ac81-bb12719752d1,HARV_033.basePlot.cfc,D01,HARV,HARV_033,tower,AVQkM2txccdJInaV60DTclWghgJyQkIgm+zcaCgWO+c=,A00000000772,AVQkM2txccdJInaV60DTclWghgJyQkIgKcu9WGq0iVY=,A00000032127,...,1,NaN,University of Wyoming Stable Isotope Facility,"010 EA-IRMS organic d15n, d13c, C% & N%",Costech 4010 Elemental Analyzer with Costech Z...,rHBrhUbTwVSLtfc5tc9S18/eJGkjVJIj,L/6ZjMlRllRQ+sQbi+7sFw==,dryingProtocolError,20211222T024324Z,RELEASE-2022
1,1f3f2937-cfc3-46d5-ba15-704ed3944829,HARV_033.basePlot.cfc,D01,HARV,HARV_033,tower,AVQkM2txcccfnFvOIWuQJKbDLFELfXIRFAGF9qDSxxY=,A00000000744,AVQkM2txcccfnFvOIWuQJKbDLFELfXIRdPLdZR/psak=,A00000032124,...,1,NaN,University of Wyoming Stable Isotope Facility,"010 EA-IRMS organic d15n, d13c, C% & N%",Costech 4010 Elemental Analyzer with Costech Z...,rHBrhUbTwVSLtfc5tc9S18/eJGkjVJIj,L/6ZjMlRllRQ+sQbi+7sFw==,dryingProtocolError,20211222T024324Z,RELEASE-2022
2,f077e103-f21b-4343-ae7f-d90c8cd69e53,HARV_033.basePlot.cfc,D01,HARV,HARV_033,tower,AVQkM2txccf8cJ5aHkcu56cKijO7cC4XXAftJL7SYdc=,A00000000773,AVQkM2txccf8cJ5aHkcu56cKijO7cC4XwiVTEeAQUH0=,A00000032120,...,1,NaN,University of Wyoming Stable Isotope Facility,"010 EA-IRMS organic d15n, d13c, C% & N%",Costech 4010 Elemental Analyzer with Costech Z...,rHBrhUbTwVSLtfc5tc9S18/eJGkjVJIj,L/6ZjMlRllRQ+sQbi+7sFw==,dryingProtocolError,20211222T024324Z,RELEASE-2022
3,c6cf3985-6e4d-4e0b-89de-1b751a96cff3,HARV_035.basePlot.cfc,D01,HARV,HARV_035,tower,AVQkM2txcceJLtxcC/BMJLp/Hy37G7o+JE+VfqIxv8Q=,A00000000788,AVQkM2txcceJLtxcC/BMJLp/Hy37G7o+HiRlP/KGqtk=,A00000032110,...,1,NaN,University of Wyoming Stable Isotope Facility,"010 EA-IRMS organic d15n, d13c, C% & N%",Costech 4010 Elemental Analyzer with Costech Z...,rHBrhUbTwVSLtfc5tc9S18/eJGkjVJIj,L/6ZjMlRllRQ+sQbi+7sFw==,dryingProtocolError,20211222T024324Z,RELEASE-2022
4,d587ea71-7d2b-4aba-a41e-fbf8cb7ed177,HARV_035.basePlot.cfc,D01,HARV,HARV_035,tower,AVQkM2txccdkxXlfsPQ38KN4nflsRnjpGaXy/aBhKBU=,A00000000745,AVQkM2txccdkxXlfsPQ38KN4nflsRnjpkkdTwPNTJtc=,A00000032112,...,1,NaN,University of Wyoming Stable Isotope Facility,"010 EA-IRMS organic d15n, d13c, C% & N%",Costech 4010 Elemental Analyzer with Costech Z...,rHBrhUbTwVSLtfc5tc9S18/eJGkjVJIj,L/6ZjMlRllRQ+sQbi+7sFw==,dryingProtocolError,20211222T024324Z,RELEASE-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,16b914cf-74f6-46eb-93ed-3a6574b03485,PUUM_034.basePlot.cfc,D20,PUUM,PUUM_034,tower,5DDwl49GZC1QV8Wxd65gxSUL9jPDIULLL9kVYlHly/M=,A00000066449,5DDwl49GZC1QV8Wxd65gxSUL9jPDIULLJLyHwncA/OM=,A00000066303,...,1,NaN,University of Wyoming Stable Isotope Facility,"010 EA-IRMS organic d15N, d13C, C% & N%",Costech 4010 Elemental Analyzer with Costech Z...,rHBrhUbTwVSLtfc5tc9S18/eJGkjVJIj,L/6ZjMlRllRQ+sQbi+7sFw==,NaN,20211221T195003Z,RELEASE-2022
416,6b615231-30e6-4527-b419-b6e0ab9f3c32,PUUM_034.basePlot.cfc,D20,PUUM,PUUM_034,tower,5DDwl49GZC2ynXd2yGS6snOjE1lPXdnxKIgVq0sD3UM=,A00000066436,5DDwl49GZC2ynXd2yGS6snOjE1lPXdnxDu7sXuI3ZWI=,A00000066265,...,1,NaN,University of Wyoming Stable Isotope Facility,"010 EA-IRMS organic d15N, d13C, C% & N%",Costech 4010 Elemental Analyzer with Costech Z...,rHBrhUbTwVSLtfc5tc9S18/eJGkjVJIj,L/6ZjMlRllRQ+sQbi+7sFw==,NaN,20211221T195003Z,RELEASE-2022
417,15e0c894-d178-400c-9988-01502a7e197e,PUUM_034.basePlot.cfc,D20,PUUM,PUUM_034,tower,5DDwl49GZC2ynXd2yGS6snOjE1lPXdnxKIgVq0sD3UM=,A00000066436,5DDwl49GZC2ynXd2yGS6snOjE1lPXdnxDu7sXuI3ZWI=,A00000066265,...,1,NaN,University of Wyoming Stable Isotope Facility,"009 EA-IRMS organic d15N, & N%",Carlo Erba Elemental Analyzer with Costech Zer...,rHBrhUbTwVSLtfc5tc9S18/eJGkjVJIj,L/6ZjMlRllRQ+sQbi+7sFw==,NaN,20211221T195003Z,RELEASE-2022
418,169a8f97-a86a-4b04-b7c6-15fa27810c09,PUUM_034.basePlot.cfc,D20,PUUM,PUUM_034,tower,5DD